In [1]:
import sys
sys.path.insert(0, '../feature_generation')
sys.path.insert(0, '../..')
import pandas as pd
import numpy as np
import keras
import models
import paths
import shutil
from PIL import Image
import math
from os.path import join
from os import listdir
from keras.preprocessing.image import ImageDataGenerator
from sklearn import model_selection
import matplotlib.pyplot as plt
import train_cnn
import test_cnn

%matplotlib inline

Using TensorFlow backend.


In [2]:
df = pd.read_json(paths.TRAIN_JSON)
listing_id_interest = df[['listing_id', 'interest_level']]

In [3]:
# only use one k fold since we're using a cnn
trn_indices = []
val_indices = []

kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
categories = ['low', 'medium', 'high']
count = 0
for dev_index, val_index in kf.split(range(len(listing_id_interest))):
    
    print "Beginning split {}".format(count)
    trn_indices = dev_index
    val_indices = val_index
    ids = listing_id_interest['listing_id']
    ids = np.array(ids)
    val_ids = ids[val_indices]
    # move everything back into train
    for folder in categories:
        for img in listdir(join(paths.VAL_IMAGES, folder)):
            shutil.move(join(paths.VAL_IMAGES,folder,img), 
                        join(paths.TRN_IMAGES,folder,img))
    # creating val set
    for folder in categories:
        for img in listdir(join(paths.TRN_IMAGES, folder)):
            listing_id = int(img[:img.index('_')])
            if listing_id in val_ids:
                shutil.move(join(paths.TRN_IMAGES,folder,img), 
                            join(paths.VAL_IMAGES,folder,img))
    print "Organized dataset successfully. Now training..."
    train_cnn.train(save_path = join(paths.CNN_OUTPUT_DIR, 
                                     "kfold_{}.h5".format(count)))
    test_cnn.test(save_id_path=join(paths.CNN_OUTPUT_DIR,
                                     "kfold_{}_ids.npy".format(count),
                   save_path = join(paths.CNN_OUTPUT_DIR,
                                     "kfold_{}_preds.npy".format(count))))
    print "Testing..."
    count += 1
"Finished!"

Beginning split 0
Organized dataset successfully. Now training...


../feature_generation/models.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu", input_shape=(224, 224,..., name="block1_conv1")`
  model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv1', input_shape=input_shape))
../feature_generation/models.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu", name="block1_conv2")`
  model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv2'))
../feature_generation/models.py:42: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same", activation="relu", name="block2_conv1")`
  model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv1'))
../feature_generation/models.py:43: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same", activat

Found 221474 images belonging to 3 classes.
Found 54957 images belonging to 3 classes.


../feature_generation/train_cnn.py:36: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, workers=1, validation_data=<keras.pre..., steps_per_epoch=216, epochs=10, callbacks=[], validation_steps=1717)`
  callbacks=[], nb_worker=1)


Epoch 1/10
215/216 [============================>.] - ETA: 0s - loss: 3.3738 - acc: 0.5343

KeyboardInterrupt: 